In [ ]:
import csv
%matplotlib inline

file = open('tess-ebs.csv')
dictReader = csv.DictReader(file)

EB_dictionary = {}
num = 0
for row in dictReader:
    if num == 10:
        break
    else:
        num += 1

    EB_dictionary.update({row['tess_id']: row})

EB_dictionary.keys()

In [ ]:
import time
import warnings
warnings.filterwarnings('ignore')
import eleanor
import matplotlib.pyplot as plt
import numpy as np
import lightkurve as lk
from transitleastsquares import transitleastsquares

plt.rcParams['font.size'] = 16

BLS_period_dict = {}
TLS_period_dict = {}

BLS_time = 0
TLS_time = 0

for key in EB_dictionary:
    try:
        star = eleanor.Source(tic=key, sector='recent')
        data = eleanor.TargetData(star, height=15, width=15, bkg_size=31, do_psf=False, do_pca=True, regressors='corner')

        print(key)
    except:
        continue
    else:
        lc = data.to_lightkurve()
        lc = lc.normalize().flatten(window_length=901)

        period = np.linspace(0.07, 25, 10000)

        start_time = time.time()
        bls = lc.to_periodogram(method='bls', period=period, frequency_factor=500, minimum_period=0.07, duration=0.06)
        end_time = time.time()

        BLS_time = end_time - start_time

        system_period = bls.period_at_max_power * 2
        # planet_b_t0 = bls.transit_time_at_max_power
        # planet_b_dur = bls.duration_at_max_power
        #
        # ax = lc.fold(period=planet_b_period, epoch_time=planet_b_t0).scatter()
        # ax.set_xlim(-1, 1)

        BLS_period_dict.update({key : system_period})

        start_time = time.time()
        model = transitleastsquares(lc.time.value, lc.flux.value)
        results = model.power(period_min=0.1, period_max=25.0)
        end_time = time.time()

        TLS_time = end_time - start_time

        TLS_period_dict.update({key : max(results.periods) * 2})



BLS_time / 10, TLS_time / 10

In [ ]:
BLS_period_dict